In [11]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

data = pd.read_stata('ccm_1985_2020_cln_stata-1.dta')

data = data[(data['fyear'] >= 2005) & (data['fyear'] <= 2016)]

data['Debt'] = (data['dltt'] + data['dlc']) / data['at']
data['Size'] = np.log(data['sale'])
data['MB'] = (data['prcc_f'] * data['cshpri'] + data['dlc'] + data['dltt'] + data['pstkl'] - data['txditc']) / data['at']
data['D_RD'] = np.where(data['xrd'].fillna(0) > 0, 1, 0)

data['RD_rank'] = pd.cut(data['xrd'] / data['sale'], bins=[-np.inf, 0, 0.25, 0.5, 0.75, np.inf], labels=[1, 2, 3, 4, 5])

data['D_Debt'] = np.where(data['Debt'] > 0.25, 1, 0)
data['D_DecIPO'] = pd.to_datetime(data['ipodate']).dt.month == 12

def winsorize(series):
    return np.clip(series, series.quantile(0.01), series.quantile(0.99))

data['Debt'] = winsorize(data['Debt'])
data['Size'] = winsorize(data['Size'])
data['MB'] = winsorize(data['MB'])

summary_stats = data[['Debt', 'Size', 'MB', 'D_RD', 'RD_rank', 'D_Debt', 'D_DecIPO']].describe(percentiles=[.25, .5, .75])
summary_stats.to_csv('summary_statistics.csv')

summary_stats_with_rd = data[data['D_RD'] == 1][['Debt', 'Size', 'MB', 'D_RD', 'RD_rank', 'D_Debt', 'D_DecIPO']].describe(percentiles=[.25, .5, .75])
summary_stats_without_rd = data[data['D_RD'] == 0][['Debt', 'Size', 'MB', 'D_RD', 'RD_rank', 'D_Debt', 'D_DecIPO']].describe(percentiles=[.25, .5, .75])
summary_stats_with_rd.to_csv('summary_statistics_with_rd.csv')
summary_stats_without_rd.to_csv('summary_statistics_without_rd.csv')

X = data[['Size', 'MB', 'D_RD']]
X = sm.add_constant(X)
y = data['Debt']

model = sm.OLS(y, X).fit()
print(model.summary())

ols_results = pd.DataFrame({
    'Variables': ['Intercept', 'Size', 'MB', 'D_RD'],
    'Coefficients': model.params,
    'Standard Errors': model.bse,
    't-values': model.tvalues,
    'p-values': model.pvalues
})


ols_results['Significance'] = ols_results['p-values'].apply(lambda p: '***' if p < 0.01 else ('**' if p < 0.05 else ('*' if p < 0.1 else '')))
ols_results.to_csv('ols_results.csv', index=False)

predicted_debt_means = model.predict(X.mean())
predicted_debt_medians = model.predict(X.median())

effect_25th_percentile = model.params['D_RD'] + model.params['MB'] * data['MB'].quantile(0.25)
effect_75th_percentile = model.params['D_RD'] + model.params['MB'] * data['MB'].quantile(0.75)

print(f'Predicted Debt (means): {predicted_debt_means}')
print(f'Predicted Debt (medians): {predicted_debt_medians}')
print(f'Effect of D(RD) at 25th percentile of MB: {effect_25th_percentile}')
print(f'Effect of D(RD) at 75th percentile of MB: {effect_75th_percentile}')

                            OLS Regression Results                            
Dep. Variable:                   Debt   R-squared:                       0.105
Model:                            OLS   Adj. R-squared:                  0.105
Method:                 Least Squares   F-statistic:                     638.3
Date:                Tue, 04 Feb 2025   Prob (F-statistic):               0.00
Time:                        22:44:09   Log-Likelihood:                 2092.5
No. Observations:               16257   AIC:                            -4177.
Df Residuals:                   16253   BIC:                            -4146.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1113      0.007     16.445      0.0

In [2]:
# Add interaction term MB * D_RD
data['MB_D_RD'] = data['MB'] * data['D_RD']

# Augmented linear regression model with interaction term
X_interaction = data[['Size', 'MB', 'D_RD', 'MB_D_RD']]
X_interaction = sm.add_constant(X_interaction)
y = data['Debt']

model_interaction = sm.OLS(y, X_interaction).fit()
print(model_interaction.summary())

# Create Table 2 for interaction model results
interaction_results = pd.DataFrame({
    'Variables': ['Intercept', 'Size', 'MB', 'D_RD', 'MB_D_RD'],
    'Coefficients': model_interaction.params,
    'Standard Errors': model_interaction.bse,
    't-values': model_interaction.tvalues,
    'p-values': model_interaction.pvalues
})

# Add significance levels
interaction_results['Significance'] = interaction_results['p-values'].apply(lambda p: '***' if p < 0.01 else ('**' if p < 0.05 else ('*' if p < 0.1 else '')))
interaction_results.to_csv('interaction_results.csv', index=False)

                            OLS Regression Results                            
Dep. Variable:                   Debt   R-squared:                       0.109
Model:                            OLS   Adj. R-squared:                  0.108
Method:                 Least Squares   F-statistic:                     495.3
Date:                Tue, 04 Feb 2025   Prob (F-statistic):               0.00
Time:                        22:31:45   Log-Likelihood:                 2122.2
No. Observations:               16257   AIC:                            -4234.
Df Residuals:                   16252   BIC:                            -4196.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1297      0.007     18.104      0.0

In [3]:
# Split-sample analysis for firms with R&D (D_RD == 1)
data_with_rd = data[data['D_RD'] == 1]
X_with_rd = data_with_rd[['Size', 'MB', 'D_RD']]
X_with_rd = sm.add_constant(X_with_rd)
y_with_rd = data_with_rd['Debt']
model_with_rd = sm.OLS(y_with_rd, X_with_rd).fit()

# Split-sample analysis for firms without R&D (D_RD == 0)
data_without_rd = data[data['D_RD'] == 0]
X_without_rd = data_without_rd[['Size', 'MB', 'D_RD']]
X_without_rd = sm.add_constant(X_without_rd)
y_without_rd = data_without_rd['Debt']
model_without_rd = sm.OLS(y_without_rd, X_without_rd).fit()

# Create Table 2 for split-sample results
split_sample_results = pd.DataFrame({
    'Variables': ['Intercept', 'Size', 'MB', 'D_RD'],
    'Coefficients (With R&D)': model_with_rd.params,
    'Standard Errors (With R&D)': model_with_rd.bse,
    'Coefficients (Without R&D)': model_without_rd.params,
    'Standard Errors (Without R&D)': model_without_rd.bse
})

# Add significance levels
split_sample_results['Significance (With R&D)'] = split_sample_results['Coefficients (With R&D)'].apply(lambda p: '***' if p < 0.01 else ('**' if p < 0.05 else ('*' if p < 0.1 else '')))
split_sample_results['Significance (Without R&D)'] = split_sample_results['Coefficients (Without R&D)'].apply(lambda p: '***' if p < 0.01 else ('**' if p < 0.05 else ('*' if p < 0.1 else '')))
split_sample_results.to_csv('split_sample_results.csv', index=False)

In [4]:
import statsmodels.formula.api as smf
from statsmodels.discrete.discrete_model import Logit, Probit

# Logistic regression model for eq. (1)
logit_model_1 = smf.logit('D_Debt ~ Size + MB + D_RD', data=data).fit()
print(logit_model_1.summary())

# Predict D(Debt) at means and medians
means = data[['Size', 'MB', 'D_RD']].mean()
medians = data[['Size', 'MB', 'D_RD']].median()

pred_means = logit_model_1.predict(pd.DataFrame([means]))
pred_medians = logit_model_1.predict(pd.DataFrame([medians]))

# Marginal effects at medians
marginal_effects = logit_model_1.get_margeff(at='median').summary()

# Marginal effects of D(RD) for firms at the 25th and 75th percentiles of MB
mb_25th = data['MB'].quantile(0.25)
mb_75th = data['MB'].quantile(0.75)

# Create a dataframe to represent the values at the 25th and 75th percentiles
atexog_25th = medians.copy()
atexog_25th['MB'] = mb_25th

atexog_75th = medians.copy()
atexog_75th['MB'] = mb_75th

# Compute the marginal effects
marginal_effects_25th = logit_model_1.get_margeff(atexog=pd.DataFrame([atexog_25th])).summary()
marginal_effects_75th = logit_model_1.get_margeff(atexog=pd.DataFrame([atexog_75th])).summary()

# Logistic regression model for eq. (2) with interaction term
data['MB_D_RD'] = data['MB'] * data['D_RD']
logit_model_2 = smf.logit('D_Debt ~ Size + MB + D_RD + MB_D_RD', data=data).fit()
print(logit_model_2.summary())

# Add interaction term to means and medians
means['MB_D_RD'] = means['MB'] * means['D_RD']
medians['MB_D_RD'] = medians['MB'] * medians['D_RD']

# Predict D(Debt) at means and medians for eq. (2)
pred_means_2 = logit_model_2.predict(pd.DataFrame([means]))
pred_medians_2 = logit_model_2.predict(pd.DataFrame([medians]))

# Marginal effects for eq. (2) at medians
marginal_effects_2 = logit_model_2.get_margeff(at='median').summary()

# Add interaction term to atexog_25th and atexog_75th
atexog_25th['MB_D_RD'] = atexog_25th['MB'] * atexog_25th['D_RD']
atexog_75th['MB_D_RD'] = atexog_75th['MB'] * atexog_75th['D_RD']

# Marginal effects of D(RD) for firms at the 25th and 75th percentiles of MB for eq. (2)
marginal_effects_25th_2 = logit_model_2.get_margeff(atexog=pd.DataFrame([atexog_25th])).summary()
marginal_effects_75th_2 = logit_model_2.get_margeff(atexog=pd.DataFrame([atexog_75th])).summary()

# Probit regression model for eq. (1)
probit_model_1 = smf.probit('D_Debt ~ Size + MB + D_RD', data=data).fit()
print(probit_model_1.summary())

# Probit regression model for eq. (2) with interaction term
probit_model_2 = smf.probit('D_Debt ~ Size + MB + D_RD + MB_D_RD', data=data).fit()
print(probit_model_2.summary())

# Custom Tobit model using censored regression
def tobit_log_likelihood(params, exog, endog, left, right):
    model = sm.OLS(endog, exog)
    llf = model.loglike(params)
    left_censored = endog <= left
    right_censored = endog >= right
    uncensored = (~left_censored & ~right_censored)
    
    llf[uncensored] = model.loglikeobs(params)
    llf[left_censored] = np.log(model.cdf(left, params))
    llf[right_censored] = np.log(1 - model.cdf(right, params))
    
    return llf.sum()

class Tobit(sm.GLM):
    def __init__(self, endog, exog, left=np.NINF, right=np.inf, **kwargs):
        self.left = left
        self.right = right
        super(Tobit, self).__init__(endog, exog, **kwargs)
        self.loglike = lambda params: tobit_log_likelihood(params, self.exog, self.endog, self.left, self.right)

# Tobit model for eq. (1) with censoring at 0 and 1
exog = sm.add_constant(data[['Size', 'MB', 'D_RD']])
endog = data['Debt']
tobit_model_1 = Tobit(endog, exog, left=0, right=1).fit()
print(tobit_model_1.summary())

# Create DataFrames for each model's results
logit_results_eq1 = pd.DataFrame({
    'Variables': ['Intercept', 'Size', 'MB', 'D_RD'],
    'Logit Coefficients (eq. 1)': logit_model_1.params,
    'Logit Std Errors (eq. 1)': logit_model_1.bse
})

logit_results_eq2 = pd.DataFrame({
    'Variables': ['Intercept', 'Size', 'MB', 'D_RD', 'MB_D_RD'],
    'Logit Coefficients (eq. 2)': logit_model_2.params,
    'Logit Std Errors (eq. 2)': logit_model_2.bse
})

probit_results_eq1 = pd.DataFrame({
    'Variables': ['Intercept', 'Size', 'MB', 'D_RD'],
    'Probit Coefficients (eq. 1)': probit_model_1.params,
    'Probit Std Errors (eq. 1)': probit_model_1.bse
})

probit_results_eq2 = pd.DataFrame({
    'Variables': ['Intercept', 'Size', 'MB', 'D_RD', 'MB_D_RD'],
    'Probit Coefficients (eq. 2)': probit_model_2.params,
    'Probit Std Errors (eq. 2)': probit_model_2.bse
})

tobit_results_eq1 = pd.DataFrame({
    'Variables': ['Intercept', 'Size', 'MB', 'D_RD'],
    'Tobit Coefficients (eq. 1)': tobit_model_1.params,
    'Tobit Std Errors (eq. 1)': tobit_model_1.bse
})

# Concatenate the results DataFrames
results = pd.concat([logit_results_eq1, logit_results_eq2, probit_results_eq1, probit_results_eq2, tobit_results_eq1], axis=1)

# Add significance levels
results['Logit Significance (eq. 1)'] = results['Logit Coefficients (eq. 1)'].apply(lambda p: '***' if p < 0.01 else ('**' if p < 0.05 else ('*' if p < 0.1 else '')))
results['Logit Significance (eq. 2)'] = results['Logit Coefficients (eq. 2)'].apply(lambda p: '***' if p < 0.01 else ('**' if p < 0.05 else ('*' if p < 0.1 else '')))
results['Probit Significance (eq. 1)'] = results['Probit Coefficients (eq. 1)'].apply(lambda p: '***' if p < 0.01 else ('**' if p < 0.05 else ('*' if p < 0.1 else '')))
results['Probit Significance (eq. 2)'] = results['Probit Coefficients (eq. 2)'].apply(lambda p: '***' if p < 0.01 else ('**' if p < 0.05 else ('*' if p < 0.1 else '')))
results['Tobit Significance (eq. 1)'] = results['Tobit Coefficients (eq. 1)'].apply(lambda p: '***' if p < 0.01 else ('**' if p < 0.05 else ('*' if p < 0.1 else '')))
results.to_csv('logit_probit_tobit_results.csv', index=False)

Optimization terminated successfully.
         Current function value: 0.584486
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                 D_Debt   No. Observations:                16257
Model:                          Logit   Df Residuals:                    16253
Method:                           MLE   Df Model:                            3
Date:                Tue, 04 Feb 2025   Pseudo R-squ.:                 0.08471
Time:                        22:31:45   Log-Likelihood:                -9502.0
converged:                       True   LL-Null:                       -10381.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.8112      0.077    -23.477      0.000      -1.962      -1.660
Size           0.2749      0.

In [5]:
# discrete examine some outputs
pred_means
marginal_effects

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:                 D_Debt
Method:                          dydx
At:                            median
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Size           0.0561      0.002     25.189      0.000       0.052       0.060
MB            -0.0226      0.003     -8.007      0.000      -0.028      -0.017
D_RD          -0.1220      0.007    -18.479      0.000      -0.135      -0.109
==============================================================================
"""

In [6]:
# discrete examine some outputs
pred_medians
marginal_effects_2

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:                 D_Debt
Method:                          dydx
At:                            median
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Size           0.0527      0.002     24.665      0.000       0.049       0.057
MB            -0.0488      0.004    -11.310      0.000      -0.057      -0.040
D_RD          -0.1812      0.009    -20.762      0.000      -0.198      -0.164
MB_D_RD        0.0423      0.005      8.391      0.000       0.032       0.052
==============================================================================
"""

In [7]:
from statsmodels.sandbox.regression.gmm import IV2SLS

# First Stage: Regress MB on the instrument (D_DecIPO) and other exogenous variables (Size, D_RD)
first_stage = smf.ols('MB ~ Size + D_RD + D_DecIPO', data=data).fit()
data['MB_hat'] = first_stage.fittedvalues

# Second Stage: Regress Debt on the predicted values of MB (MB_hat) and other exogenous variables (Size, D_RD)
second_stage = smf.ols('Debt ~ Size + D_RD + MB_hat', data=data).fit()

# OLS estimation of eq. (1) for comparison
ols_model = smf.ols('Debt ~ Size + MB + D_RD', data=data).fit()

# Create Table 4 for the results of both stages of 2-SLS
table_4 = pd.DataFrame({
    'First Stage Coefficients': first_stage.params,
    'First Stage Std Errors': first_stage.bse,
    'Second Stage Coefficients': second_stage.params,
    'Second Stage Std Errors': second_stage.bse
})

# Add significance levels
table_4['First Stage Significance'] = table_4['First Stage Coefficients'].apply(lambda p: '***' if p < 0.01 else ('**' if p < 0.05 else ('*' if p < 0.1 else '')))
table_4['Second Stage Significance'] = table_4['Second Stage Coefficients'].apply(lambda p: '***' if p < 0.01 else ('**' if p < 0.05 else ('*' if p < 0.1 else '')))

# Print the results
print("Table 4: 2-SLS Regression Results")
print(table_4)

# Comparison to OLS estimation of eq. (1) (Table 2)
table_2 = pd.DataFrame({
    'OLS Coefficients': ols_model.params,
    'OLS Std Errors': ols_model.bse
})

# Add significance levels
table_2['OLS Significance'] = table_2['OLS Coefficients'].apply(lambda p: '***' if p < 0.01 else ('**' if p < 0.05 else ('*' if p < 0.1 else '')))

# Print the OLS results
print("Table 2: OLS Estimation of eq. (1)")
print(table_2)


Table 4: 2-SLS Regression Results
                  First Stage Coefficients  First Stage Std Errors  \
D_DecIPO[T.True]                 -0.223022                0.041126   
D_RD                              0.524596                0.025317   
Intercept                         2.284175                0.044236   
MB_hat                                 NaN                     NaN   
Size                             -0.113710                0.006058   

                  Second Stage Coefficients  Second Stage Std Errors  \
D_DecIPO[T.True]                        NaN                      NaN   
D_RD                              -0.044484                 0.014383   
Intercept                          0.250349                 0.059782   
MB_hat                            -0.064781                 0.026285   
Size                               0.016735                 0.003113   

                 First Stage Significance Second Stage Significance  
D_DecIPO[T.True]                      *** 

In [ ]:
# place holder for the results

In [17]:
from linearmodels.panel import PanelOLS, RandomEffects

# Set the index for panel data
data_panel = data.set_index(['gvkey', 'fyear'])

# Firm-Level Fixed Effects (FE)
fe_model = PanelOLS.from_formula('Debt ~ 1 + Size + MB + D_RD + EntityEffects', data=data_panel, drop_absorbed=True).fit()
print(fe_model.summary)

# Firm-Level Random Effects (RE)
re_model = RandomEffects.from_formula('Debt ~ 1 + Size + MB + D_RD', data=data_panel).fit()
print(re_model.summary)

# Firm-Level Fixed Effects with Year Fixed Effects
fe_year_model = PanelOLS.from_formula('Debt ~ 1 + Size + MB + D_RD + EntityEffects + TimeEffects', data=data_panel, drop_absorbed=True).fit()
print(fe_year_model.summary)

# Firm-Level Random Effects with Year Fixed Effects
re_year_model = RandomEffects.from_formula('Debt ~ 1 + Size + MB + D_RD + TimeEffects', data=data_panel).fit()
print(re_year_model.summary)

# Augmented Model with D(DecIPO) as an additional explanatory variable
fe_augmented_model = PanelOLS.from_formula('Debt ~ 1 + Size + MB + D_RD + D_DecIPO + EntityEffects', data=data_panel, drop_absorbed=True).fit()
print(fe_augmented_model.summary)

re_augmented_model = RandomEffects.from_formula('Debt ~ 1 + Size + MB + D_RD + D_DecIPO', data=data_panel).fit()
print(re_augmented_model.summary)

# OLS estimation of eq. (1) for comparison
ols_model = smf.ols('Debt ~ Size + MB + D_RD', data=data_panel.reset_index()).fit()

# Function to format the parameters
def format_params(model, variables):
    params = model.params
    bse = model.bse if hasattr(model, 'bse') else model.std_errors
    pvalues = model.pvalues

    formatted_params = {
    var: (
        f'{params[var]:.3f}'
        f'{"***" if pvalues[var] < 0.01 else ("**" if pvalues[var] < 0.05 else ("*" if pvalues[var] < 0.1 else ""))}\n'
        f'({bse[var]:.3f})'
    ) if var in params else ''
    for var in variables
}
    return formatted_params

variables = ['Intercept', 'Size', 'MB', 'D_RD', 'D_DecIPO']

params_ols = format_params(ols_model, variables)
params_fe = format_params(fe_model, variables)
params_re = format_params(re_model, variables)
params_fe_year = format_params(fe_year_model, variables)
params_re_year = format_params(re_year_model, variables)
params_fe_augmented = format_params(fe_augmented_model, variables)
params_re_augmented = format_params(re_augmented_model, variables)

table_5 = pd.DataFrame({
    'Variables': variables,
    'OLS': [params_ols[var] for var in variables],
    'FE': [params_fe[var] for var in variables],
    'RE': [params_re[var] for var in variables],
    'FE Year': [params_fe_year[var] for var in variables],
    'RE Year': [params_re_year[var] for var in variables],
    'FE Augmented': [params_fe_augmented[var] for var in variables],
    'RE Augmented': [params_re_augmented[var] for var in variables]
})

# Save the results to a CSV file
table_5.to_csv('panel_data_estimations_table_5.csv', index=False)

# Print the results
print("Table 5: Panel Data Estimations (Fixed Effects and Random Effects)")
print(table_5)


                          PanelOLS Estimation Summary                           
Dep. Variable:                   Debt   R-squared:                        0.0176
Estimator:                   PanelOLS   R-squared (Between):              0.1278
No. Observations:               16257   R-squared (Within):               0.0176
Date:                Tue, Feb 04 2025   R-squared (Overall):              0.0949
Time:                        22:58:01   Log-likelihood                 1.365e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      82.393
Entities:                        2467   P-value                           0.0000
Avg Obs:                       6.5898   Distribution:                 F(3,13787)
Min Obs:                       1.0000                                           
Max Obs:                       12.000   F-statistic (robust):             82.393
                            

/var/folders/53/z0kd2hl56_1g6ntqlk8pfcgm0000gn/T/ipykernel_61414/1376311673.py:23: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

D_DecIPO

  fe_augmented_model = PanelOLS.from_formula('Debt ~ 1 + Size + MB + D_RD + D_DecIPO + EntityEffects', data=data_panel, drop_absorbed=True).fit()


                          PanelOLS Estimation Summary                           
Dep. Variable:                   Debt   R-squared:                        0.0176
Estimator:                   PanelOLS   R-squared (Between):              0.1278
No. Observations:               16257   R-squared (Within):               0.0176
Date:                Tue, Feb 04 2025   R-squared (Overall):              0.0949
Time:                        22:58:02   Log-likelihood                 1.365e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      82.393
Entities:                        2467   P-value                           0.0000
Avg Obs:                       6.5898   Distribution:                 F(3,13787)
Min Obs:                       1.0000                                           
Max Obs:                       12.000   F-statistic (robust):             82.393
                            

In [9]:
data

sic state    ipodate  lpermno                conm  exchg  \
gvkey  fyear                                                               
001004 2005.0  5080    IL 1972-04-24  54594.0            AAR CORP   11.0   
       2006.0  5080    IL 1972-04-24  54594.0            AAR CORP   11.0   
       2007.0  5080    IL 1972-04-24  54594.0            AAR CORP   11.0   
       2008.0  5080    IL 1972-04-24  54594.0            AAR CORP   11.0   
       2009.0  5080    IL 1972-04-24  54594.0            AAR CORP   11.0   
...             ...   ...        ...      ...                 ...    ...   
287252 2016.0  7370    FL 2007-11-12  14075.0  MIX TELEMATICS LTD   11.0   
311524 2013.0  2860    PA 2013-04-18  13861.0        TAMINCO CORP   11.0   
317264 2014.0  4412    CT 2014-05-08  14642.0      DORIAN LPG LTD   11.0   
       2015.0  4412    CT 2014-05-08  14642.0      DORIAN LPG LTD   11.0   
       2016.0  4412    CT 2014-05-08  14642.0      DORIAN LPG LTD   11.0   

                   act        at      dlc     dltt  ...  prcc_f      Debt  \
gvkey  fyear                                        ...                     
001004 2005.0  624.454   978.819    2.289  318.576  ...   24.08  0.327808   
       2006.0  645.721  1067.633   74.245  253.611  ...   32.50  0.307087   
       2007.0  783.431  1362.010   22.994  507.918  ...   19.28  0.389800   
       2008.0  851.312  1377.511   63.600  392.984  ...   14.70  0.331456   
       2009.0  863.429  1501.042  100.833  336.191  ...   19.70  0.291147   
...                ...       ...      ...      ...  ...     ...       ...   
287252 2016.0   52.339   142.052    1.449    0.000  ...    6.30  0.010200   
311524 2013.0  311.000  1856.000    7.000  908.000  ...   20.21  0.492996   
317264 2014.0  233.211  1099.101   15.678  184.666  ...   13.03  0.182280   
       2015.0  105.559  1865.926   66.265  770.103  ...    9.40  0.448232   
       2016.0   63.971  1746.235   65.979  683.985  ...   10.53  0.429475   

                   Size        MB  D_RD  RD_rank  D_Debt  D_DecIPO   MB_D_RD  \
gvkey  fyear                                                                   
001004 2005.0  6.799372  1.126777     0        1       1     False  0.000000   
       2006.0  6.967126  1.377231     0        1       1     False  0.000000   
       2007.0  7.233397  0.895736     0        1       1     False  0.000000   
       2008.0  7.261208  0.708371     0        1       1     False  0.000000   
       2009.0  7.209452  0.754059     0        1       1     False  0.000000   
...                 ...       ...   ...      ...     ...       ...       ...   
287252 2016.0  4.742643  1.074673     1        2       0     False  1.074673   
311524 2013.0  7.090077  1.077520     1        2       1     False  1.077520   
317264 2014.0  4.646533  0.858817     0        1       0     False  0.000000   
       2015.0  5.673849  0.733659     0        1       1     False  0.000000   
       2016.0  5.134963  0.755577     0        1       1     False  0.000000   

                 MB_hat  
gvkey  fyear             
001004 2005.0  1.511017  
       2006.0  1.491941  
       2007.0  1.461663  
       2008.0  1.458501  
       2009.0  1.464386  
...                 ...  
287252 2016.0  2.269484  
311524 2013.0  2.002556  
317264 2014.0  1.755816  
       2015.0  1.639000  
       2016.0  1.700277  

[16257 rows x 31 columns]